In [1]:
print('intelligibility eval')

intelligibility eval


In [ ]:
!pip install transformers
!pip install torch
!pip install jiwer
!pip install pandas==1.1.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 3.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3


In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

#model_id = "nilc-nlp/distil-whisper-coraa-mupe-asr"
model_id = "sidleal/distil-whisper-tarsila-asr-v1-200k"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    #model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    model_id, torch_dtype=torch_dtype, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    torch_dtype=torch_dtype,
    return_timestamps=True,
    device=device,
)

def trancript_with_distil(file_path):
    result = pipe(file_path)
    return result["text"]

/home/sidleal/anaconda3/envs/fs2/lib/python3.8/site-packages/transformers/pipelines/automatic_speech_recognition.py:312: FutureWarning: `max_new_tokens` is deprecated and will be removed in version 4.49 of Transformers. To remove this warning, pass `max_new_tokens` as a key inside `generate_kwargs` instead.
  warnings.warn(


In [4]:
result = trancript_with_distil("/home/sidleal/sources/FastSpeech2/output/result/cmltts_entoa_prossegue/600_outra televisão de araraquara.wav")
print("----------------------------------------")
print(result)
print("----------------------------------------")

/home/sidleal/anaconda3/envs/fs2/lib/python3.8/site-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


----------------------------------------
 outra televisão dera aquar
----------------------------------------


In [5]:
import re
import jiwer
import jiwer.transforms as tr

cer_transform = tr.Compose(
    [
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.ReduceToListOfListOfChars(),
    ]
)

# It's the jiwer default transform
wer_transform = jiwer.Compose([
    jiwer.ToLowerCase(),
    jiwer.RemoveMultipleSpaces(),
    jiwer.Strip(),
    jiwer.ReduceToListOfListOfWords(),
])

def compute_cer(reference, hypothesis):
    reference = reference.lower()
    hypothesis = hypothesis.lower()
    cer = jiwer.wer(reference, hypothesis, truth_transform=cer_transform, hypothesis_transform=cer_transform)
    return cer

def compute_wer(reference, hypothesis):
    reference = reference.lower()
    hypothesis = hypothesis.lower()
    wer = jiwer.wer(reference, hypothesis, truth_transform=wer_transform, hypothesis_transform=wer_transform)
    return wer


alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZÇÃÀÁÂÊÉÍÓÔÕÚÛabcdefghijklmnopqrstuvwxyzçãàáâêéíóôõũúû1234567890%\\-\n/\\ "


def replace_special_tokens_and_normalize(text):
    text = text.lower()

    map_words = {
        "éh": "eh",
        "ehm": "eh",
        "ehn": "eh",
        "hum": "uh",
        "hm": "uh",
        "uhm": "uh",
        "hã": "ah",
        "ãh": "ah",
        "ã":  "ah",
        "hmm": "uh",
        "mm": "uh",
        "mhm": "uh"
    }

    text = re.sub("h+", "h", text)
    text = re.sub("[^{}]".format(alphabet+" "), " ", text)
    text = re.sub("[ ]+", " ", text)

    words = text.split(' ')
    new_words = []
    for word in words:
        if word == '' or word == ' ':
            continue
        if word in map_words:
            new_words.append(map_words[word])
        else:
            new_words.append(word)

    return " ".join(new_words)


def calculate_wer_cer(reference, hypothesis):
    if reference.strip() == '' or hypothesis.strip() == '':
        return 1, 1
    wer = compute_wer(reference, hypothesis)
    cer = compute_cer(reference, hypothesis)
    return wer, cer



In [5]:
#sintetizar
import pandas as pd
import subprocess

file_path = "/home/sidleal/sources/entoa-tts/experiments/sp_234_l1_transc_cer_wer.csv"
df = pd.read_csv(file_path)

for index, row in df.iterrows():
    print(f"Row {index}: {row['human']}")
    human_value = row['human']
    command_to_execute = f"cd /home/sidleal/sources/FastSpeech2 && python synthesize.py --text '{human_value}' --speaker_id 13 --restore_step 600000 --mode single -p config/cmltts_entoa_prossegue/preprocess.yaml -m config/cmltts_entoa_prossegue/model.yaml -t config/cmltts_entoa_prossegue/train.yaml"

    print(f"[{index}] Running: {command_to_execute}")

    result = subprocess.run(
        command_to_execute,
        shell=True, # We use shell=True because the command uses shell features (like passing arguments)
        capture_output=True,
        text=True,
        check=True
    )
    print(result.stdout.strip())

    break

Row 0: bom um bom artista é o que desempenha o papel na peça de acordo com o que ele está está fazendo eu acho que o que ele está o papel que ele está desempenhando ah que eles eh a gente perceba que que realmente ele está trabalhando bem sei lá não sei tem tantos bons artistas
[0] Running: cd /home/sidleal/sources/FastSpeech2 && python synthesize.py --text 'bom um bom artista é o que desempenha o papel na peça de acordo com o que ele está está fazendo eu acho que o que ele está o papel que ele está desempenhando ah que eles eh a gente perceba que que realmente ele está trabalhando bem sei lá não sei tem tantos bons artistas' --speaker_id 13 --restore_step 600000 --mode single -p config/cmltts_entoa_prossegue/preprocess.yaml -m config/cmltts_entoa_prossegue/model.yaml -t config/cmltts_entoa_prossegue/train.yaml


CalledProcessError: Command 'cd /home/sidleal/sources/FastSpeech2 && python synthesize.py --text 'bom um bom artista é o que desempenha o papel na peça de acordo com o que ele está está fazendo eu acho que o que ele está o papel que ele está desempenhando ah que eles eh a gente perceba que que realmente ele está trabalhando bem sei lá não sei tem tantos bons artistas' --speaker_id 13 --restore_step 600000 --mode single -p config/cmltts_entoa_prossegue/preprocess.yaml -m config/cmltts_entoa_prossegue/model.yaml -t config/cmltts_entoa_prossegue/train.yaml' returned non-zero exit status 1.

In [ ]:
import os
import csv

file_name = 'prossegue_cer_wer_output.csv'

folder_path = '/home/sidleal/sources/FastSpeech2/output/result/cmltts_entoa_prossegue/sp_234'

with open(file_name, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    csv_data = ["audio_name", "human_text", "human_text_norm", "transcript_original", "transcript_synthesized", "cer_original", "cer_synthesized", "wer_original", "wer_synthesized"]
    writer.writerow(csv_data)

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            audio_name = filename[:-8]
            original_audio_name = filename[:-4]
            synth_audio_name = filename[:-4] + "_synthesized.wav"

            with open(f"{folder_path}/{filename}", encoding='utf-8') as f:
                text = f.read()
                text_norm = replace_special_tokens_and_normalize(text)

                transcript_ori = trancript_with_distil(f"{folder_path}/{original_audio_name}")
                transcript_syn = trancript_with_distil(f"{folder_path}/{synth_audio_name}")

                transcript_ori_norm = replace_special_tokens_and_normalize(transcript_ori)
                transcript_syn_norm = replace_special_tokens_and_normalize(transcript_syn)

                ori_wer_cer = calculate_wer_cer(text_norm, transcript_ori_norm)
                syn_wer_cer = calculate_wer_cer(text_norm, transcript_syn_norm)

                print("--------------------------------")
                print(audio_name)
                print("-", text)
                print("-", text_norm)
                print("-", original_audio_name)
                print("-", synth_audio_name)
                print("-", transcript_ori_norm)
                print("-", transcript_syn_norm)
                print(ori_wer_cer)
                print(syn_wer_cer)

                csv_data = [audio_name, text, text_norm, transcript_ori_norm, transcript_syn_norm, ori_wer_cer[1], syn_wer_cer[1], ori_wer_cer[0], syn_wer_cer[0]]
                writer.writerow(csv_data)

            #break



In [ ]:
from scipy import stats
import pandas as pd

df = pd.read_csv(file_name)

t_statistic, p_value = stats.ttest_ind(df["wer_original"], df["wer_synthesized"])

print(f"WER ORIGINAL: {df["wer_original"].mean()*100:.2f}")
print(f"WER SINTETIZADO: {df["wer_synthesized"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})")
print(f"P-value: {p_value:.4f} ({p_value})" )

WER ORIGINAL: 13.70
WER SINTETIZADO: 23.86
T-statistic: -6.1082 (-6.108212784547283)
P-value: 0.0000 (5.242308079569724e-09)


# f5tts

In [1]:
import pandas as pd

file_name = 'cer_wer_output.csv'

df = pd.read_csv(file_name)

df.head()

,audio_name,human_text,human_text_norm,transcript_original,transcript_synthesized,cer_original,cer_synthesized,wer_original,wer_synthesized
0,sr16k_mono_0027-CP011_1250.69-1261.3,"Ô Jorge, é o seguinte, a gente esqueceu de avi...",ô jorge é o seguinte a gente esqueceu de avisa...,o jorge é o seguinte a gente esqueceu de avisa...,eu já é o seguinte a gente esqueceu de avisar ...,0.037267,0.211180,0.161290,0.387097
1,sr16k_mono_0038-CP015_950.037-979.634,E eu gostaria de lembrar que um dos motivos da...,e eu gostaria de lembrar que um dos motivos da...,eu gostaria de lembrar que um dos motivos dess...,eu gostaria de lembrar que os motivos da da re...,0.104623,0.085158,0.184211,0.118421
2,sr16k_mono_0067-CP546_1576.56-1591.05,"É. é que Veja bem, essa experiência, eu gostar...",é é que veja bem essa experiência eu gostaria ...,é eh que veja bem essa experiência eu gostaria...,é que veja bem essa experiência eu gostaria qu...,0.102041,0.073469,0.170213,0.212766
3,sr16k_mono_0082-CP707_1508.09-1518.64,"Mas eu gostaria... Claudio, dá dá só uma uma u...",mas eu gostaria claudio dá dá só uma uma uma a...,mas eu gostaria caldo dá dá só uma uma aperfei...,você gostaria de é de dada só uma uma aperfeiç...,0.112583,0.172185,0.214286,0.321429
4,sr16k_mono_0079-CP623_853.479-861.621,Foi nesse contraponto que eu tentei compreende...,foi nesse contraponto que eu tentei compreende...,então foi nesse contraponto que eu tentei comp...,foi nesse contrário conto que eu tentei compre...,0.148148,0.194444,0.263158,0.421053


In [ ]:
import os
import csv
import pandas as pd


df = pd.read_csv('cer_wer_output.csv')

file_name = 'cer_wer_output_v2.csv'

folder_path = 'resulting_audios_cv-fn'

with open(file_name, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    csv_data = ["audio_name", "human_text", "human_text_norm", "transcript_original", "transcript_synthesized_yourtts", "transcript_synthesized_f5tts_cv", "cer_original", "cer_synthesized_yourtts", "cer_synthesized_f5tts_cv", "wer_original", "wer_synthesized_yourtts", "wer_synthesized_f5tts_cv"]
    writer.writerow(csv_data)

    for filename in os.listdir(folder_path):
        if filename.endswith(".wav"):
            audio_name = filename[:-4]
            print(audio_name)

            audio_key = f"sr16k_mono_{audio_name[6:]}"
            print(audio_key)

            row = df.query(f"audio_name == '{audio_key}'")
            print(row['audio_name'].iloc[0])
            print(row['human_text_norm'].iloc[0])
            print("-------")
            
            text_norm = replace_special_tokens_and_normalize(row['human_text_norm'].iloc[0])

            transcript_f5cv = trancript_with_distil(f"{folder_path}/{filename}")
            transcript_f5cv_norm = replace_special_tokens_and_normalize(transcript_f5cv)

            f5cv_wer_cer = calculate_wer_cer(text_norm, transcript_f5cv_norm)

            print("-", text_norm)
            print("-", transcript_f5cv_norm)
            print(f5cv_wer_cer)

            csv_data = [audio_key, row["human_text"].iloc[0], row["human_text_norm"].iloc[0], row["transcript_original"].iloc[0], row["transcript_synthesized"].iloc[0], transcript_f5cv_norm, row["cer_original"].iloc[0], row["cer_synthesized"].iloc[0], f5cv_wer_cer[1], row["wer_original"].iloc[0], row["wer_synthesized"].iloc[0], f5cv_wer_cer[0]]
            writer.writerow(csv_data)

            #break


In [23]:
from scipy import stats
import pandas as pd

file_name = 'cer_wer_output_v2.csv'

df = pd.read_csv(file_name)

t_statistic, p_value = stats.ttest_ind(df["wer_original"], df["wer_synthesized_f5tts_cv"])

print(f"WER ORIGINAL: {df["wer_original"].mean()*100:.2f}")
print(f"WER SINTETIZADO F5 CV: {df["wer_synthesized_f5tts_cv"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})")
print(f"P-value: {p_value:.4f} ({p_value})" )

WER ORIGINAL: 13.60
WER SINTETIZADO F5 CV: 18.12
T-statistic: -3.0251 (-3.025088762473149)
P-value: 0.0028 (0.0028187078083642975)


In [24]:
from scipy import stats
import pandas as pd

file_name = 'cer_wer_output_v2.csv'

df = pd.read_csv(file_name)

t_statistic, p_value = stats.ttest_ind(df["wer_synthesized_yourtts"], df["wer_synthesized_f5tts_cv"])

print(f"WER ORIGINAL: {df["wer_synthesized_yourtts"].mean()*100:.2f}")
print(f"WER SINTETIZADO F5 CV: {df["wer_synthesized_f5tts_cv"].mean()*100:.2f}")

print(f"T-statistic: {t_statistic:.4f} ({t_statistic})")
print(f"P-value: {p_value:.4f} ({p_value})" )

WER ORIGINAL: 23.77
WER SINTETIZADO F5 CV: 18.12
T-statistic: 3.6120 (3.6120189458795777)
P-value: 0.0004 (0.0003859933928752929)
